# Final Report

The dataset contains 32 columns. Each column corresponds to a specific attribute for each participant. The level of consumption for each of the :
* ID 
* Age (18 to 65+)
* Gender (male or female)
* Education (level of education)
    + Left school before 16 years
    + Left school at 16 years
    + Left school at 17 years
    + Left school at 18 years
    + Some college or university, no certificate or degree
    + Professional certificate/ diploma 
    + University degree
    + Masters degree
    + Doctorate degree
- Country (country of residence)
    + Australia
    + Canada
    + New Zealand
    + Other
    + Republic of Ireland
    + United Kingdom
    + United States of America
- Ethnicity (ethnicity)
    + Asian
    + Black
    + Mixed-Black/Asian
    + Mixed-White/Asian
    + Mixed-White/Black
    + Other
    + White
- Nscore (measurement of neuroticism)
- Escore (measurement of extraversion)
- Oscore (openness to experience)
- Ascore (measurement of agreeableness)
- Cscore (measurement of conscientiousness)
- Impulsive (measurement of impulsiveness)
- SS (measurement of sensation seeking)
- Alcohol (level alcohol consumption)
- Amphet (level of amphetamines consumption)
- Amyl (level of amyl nitrite consumption)
- Benzos (level of benzodiazepine consumption)
- Caff (level of caffeine consumption)
- Cannabis (level of cannabis consumption)
- Choc (level of chocolate consumption)
- Coke (level of cocaine consumption)
- Crack (level of crack consumption)
- Ecstasy(level of ecstasy consumption)
- Heroin (level of heroin consumption)
- Ketamine (level of ketamine consumption)
- Legalh (level of legal highs consumption) 
- LSD is (level of LSD consumption)
- Meth (level of methadone consumption) 
- Mushrooms (level of magic mushrooms consumption) 
- Nicotine (level of nicotine consumption)
- Semer (level of fictitious drug Semeron consumption) 
- VSA (level of volatile substance abuse) 